In [ ]:
# # Install required packages for YOLOv5 and model quantization
# !pip install torch torchvision torchaudio
# !pip install yolov5
# !pip install torch==1.9.0  # Make sure compatibility with PyTorch and TensorRT (Optional)


In [4]:
# !git clone https://github.com/ultralytics/ultralytics.git
# %cd ultralytics
# !pip install -e .
# ! pip install kaggle


In [9]:
# ! pip install pandas matplotlib scikit-learn
#

In [8]:
# ! kaggle datasets download -d andrewmvd/face-mask-detection


In [ ]:
import os
import xml.etree.ElementTree as ET
import shutil
from sklearn.model_selection import train_test_split
import zipfile

In [9]:
# Unzip the dataset
zip_file_path = "face-mask-detection.zip"  # Path to the downloaded zip file
extract_to = "./face-mask-detection-dataset/"  # Target folder for extraction

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Dataset extracted to {extract_to}")


Dataset extracted to ./face-mask-detection-dataset/


In [ ]:
# Directories
BASE_DIR = "ultralytics/face-mask-detection-dataset/"
IMAGES_DIR = os.path.join(BASE_DIR, "images")
ANNOTATIONS_DIR = os.path.join(BASE_DIR, "annotations")

# Create output directories
OUTPUT_DIR = "datasets/"
os.makedirs(os.path.join(OUTPUT_DIR, "train/images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val/images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "test/images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val/labels"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "test/labels"), exist_ok=True)

# Mapping class names to IDs
CLASS_MAPPING = {"with_mask": 0, "without_mask": 1, "mask_weared_incorrect": 2}

# Parse annotations
def parse_annotations():
    labels = []
    for file_name in os.listdir(ANNOTATIONS_DIR):
        if file_name.endswith(".xml"):
            tree = ET.parse(os.path.join(ANNOTATIONS_DIR, file_name))
            root = tree.getroot()
            for obj in root.findall('object'):
                image_name = root.find('filename').text
                category = obj.find('name').text
                bbox = obj.find('bndbox')
                bbox = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                        int(bbox.find('xmax').text) - int(bbox.find('xmin').text),
                        int(bbox.find('ymax').text) - int(bbox.find('ymin').text)]
                labels.append({
                    "image": os.path.join(IMAGES_DIR, image_name),
                    "class_id": CLASS_MAPPING[category],
                    "bbox": bbox
                })
    return labels

# Convert to YOLO format
def convert_bbox_to_yolo_format(width, height, bbox):
    x_center = (bbox[0] + bbox[2] / 2) / width
    y_center = (bbox[1] + bbox[3] / 2) / height
    bbox_width = bbox[2] / width
    bbox_height = bbox[3] / height
    return x_center, y_center, bbox_width, bbox_height

# Get image and label data
labels = parse_annotations()
image_paths = [os.path.join(IMAGES_DIR, f) for f in os.listdir(IMAGES_DIR) if f.endswith('.png')]

# Split data
train_images, test_images = train_test_split(image_paths, test_size=0.2, random_state=42)
val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

# Distribute files into folders
def save_images_and_labels(image_list, split):
    for image_path in image_list:
        # Copy image
        img_name = os.path.basename(image_path)
        shutil.copy(image_path, os.path.join(OUTPUT_DIR, f"{split}/images/{img_name}"))

        # Process associated labels
        img_labels = [item for item in labels if item["image"] == image_path]
        if img_labels:
            # Get image dimensions from the XML file
            tree = ET.parse(os.path.join(ANNOTATIONS_DIR, img_name.replace('.png', '.xml')))
            root = tree.getroot()
            width = int(root.find('size/width').text)
            height = int(root.find('size/height').text)
            
            label_file_path = os.path.join(OUTPUT_DIR, f"{split}/labels/{img_name.replace('.png', '.txt')}")
            with open(label_file_path, "w") as f:
                for label in img_labels:
                    bbox_yolo = convert_bbox_to_yolo_format(width, height, label["bbox"])
                    f.write(f"{label['class_id']} " + " ".join(map(str, bbox_yolo)) + "\n")
            print(f"Labels saved to {label_file_path}")
        else:
            print(f"No labels found for {img_name}")

# Save images and labels
save_images_and_labels(train_images, "train")
save_images_and_labels(val_images, "val")
save_images_and_labels(test_images, "test")



Labels saved to datasets/train/labels/maksssksksss806.txt
Labels saved to datasets/train/labels/maksssksksss19.txt
Labels saved to datasets/train/labels/maksssksksss808.txt
Labels saved to datasets/train/labels/maksssksksss576.txt
Labels saved to datasets/train/labels/maksssksksss486.txt
Labels saved to datasets/train/labels/maksssksksss499.txt
Labels saved to datasets/train/labels/maksssksksss10.txt
Labels saved to datasets/train/labels/maksssksksss271.txt
Labels saved to datasets/train/labels/maksssksksss514.txt
Labels saved to datasets/train/labels/maksssksksss546.txt
Labels saved to datasets/train/labels/maksssksksss489.txt
Labels saved to datasets/train/labels/maksssksksss784.txt
Labels saved to datasets/train/labels/maksssksksss535.txt
Labels saved to datasets/train/labels/maksssksksss814.txt
Labels saved to datasets/train/labels/maksssksksss589.txt
Labels saved to datasets/train/labels/maksssksksss397.txt
Labels saved to datasets/train/labels/maksssksksss55.txt
Labels saved to d

In [28]:
# os.chdir("E:/Face-mask-detection")
# os.getcwd()


'E:\\Face-mask-detection'

In [56]:
# Create YAML config
yaml_content = f"""
train: {os.path.abspath(OUTPUT_DIR + '/train/images')}
val: {os.path.abspath(OUTPUT_DIR + '/val/images')}
test: {os.path.abspath(OUTPUT_DIR + '/test/images')}

nc: {len(CLASS_MAPPING)}  # Number of classes
names: {list(CLASS_MAPPING.keys())}  # Class names
"""

with open(os.path.join(OUTPUT_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("YOLOv8 data preparation completed. Check the output directory!")


YOLOv8 data preparation completed. Check the output directory!
